In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import math
import ast
from unicodedata import normalize
from collections import Counter

In [2]:
gabarito = pd.read_csv('gabarito/gabarito.csv')
dados = pd.read_csv('data/estadao_noticias_eleicao.csv')

# Função para converter string em lista
Essa função converta uma string que tem o formato de uma lista em um objeto do tipo lista.

In [3]:
def convert_str_in_lst(lista):
    return ast.literal_eval(lista)

# Convertendo células do dataframe do gabarito
Todas as células do dataframe precisaram ser convertidas, pois ao serem lidas estão no formato de string e não de lista como deveria ser, assim, fiz a conversão das strings em listas.

In [4]:
gabarito.google = gabarito.google.apply(convert_str_in_lst)
gabarito.busca_binaria = gabarito.busca_binaria.apply(convert_str_in_lst)
gabarito.tf = gabarito.tf.apply(convert_str_in_lst)
gabarito.tfidf = gabarito.tfidf.apply(convert_str_in_lst)
gabarito.bm25 = gabarito.bm25.apply(convert_str_in_lst)


# Removendo NAN
Alguns documentos possuem valores NAN no conteúdo da célula no dataframe, isso prejudica na hora de gerar os tokens para criar o índice. Portanto, substitui todas as células do dataframe que tinha NAN por ''.

In [5]:
dados = dados.replace(np.nan, '', regex=True)

# Função para limpar o texto
Essa Função remove todos os caracteres especiais do texto bem como sua acentuação.

In [6]:
def limpar_texto(texto):
    pattern = re.compile('[^a-zA-Z0-9 ]')
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return pattern.sub(' ', texto)

# Join do conteúdo
Juntando os títulos das notícias com seus respectivos subtítulos e conteúdos, e também removendo das nóticias os caracteres especiais e a acentuação para posteriomente facilitar a tokenização.

In [7]:
materias = dados.titulo + " " + dados.subTitulo +  " " + dados.conteudo
materias = materias.apply(lambda texto: limpar_texto(texto).lower())
ids = dados.idNoticia
    

# Tokenizando conteúdo
Criando tokens com cada palavra do texto para que posteriormente possam ser indexadas e associadas aos respectivos ids das notícias, e contando a frequência de cada termo no texto.

In [8]:
tokens = materias.apply(nltk.word_tokenize)
term_frequence = tokens.apply(Counter)

# Indexando tokens
Criando indices invertidos com os tokens para poder aplicar os métodos de busca.

In [9]:
index = {}

for i in range(len(tokens)):
    id_noticia = ids[i]
    palavras = tokens[i]
    for palavra in palavras:
        palavra = palavra.lower()
        if palavra not in index:
            index[palavra] = {}
        
        id_rec = index[palavra].get(id_noticia)
        
        if not id_rec:
            docs = index[palavra]
            docs[id_noticia] = term_frequence[i][palavra]

# Método que gera um dicionário com vetores de pesos
Este método gera uma dicionário cujas chaves são os ids documentos que contém os termos pesquisados e os valores são vetores que contem o peso de cada termo nos documentos, esse vetor pode ser binário ou por TF (Term Frequence).

In [10]:
def gera_docs_peso(frase, gerador_peso):
    termos = frase.split(" ")
    docs_peso = {}
    
    for i in range(len(termos)):
        termo = termos[i]
        docs = index[termo]
        for doc_id in docs:
            tf = docs[doc_id]
            
            if doc_id not in docs_peso:
                docs_peso[doc_id] = np.array([0 if j != i else gerador_peso(tf) for j in range(len(termos))])
            else:
                doc_vector = docs_peso[doc_id]
                docs_peso[doc_id] = np.array([doc_vector[j] if j != i else gerador_peso(tf) for j in range(len(termos))])
    
    return docs_peso


# Método que gera um dicionário com vetores binários da consulta
Este método gera uma dicionário cujas chaves são os ids documentos que contém os termos pesquisados e os valores são vetores que contem o peso binário de cada termo nos documentos, usando como método base o método gera_docs_peso.

In [11]:
def gera_vetor_binario(frase):
    def gerador_peso(tf):
        return 1
    return gera_docs_peso(frase, gerador_peso)

# Método que gera um dicionário com vetores TF da consulta
Este método gera uma dicionário cujas chaves são os ids documentos que contém os termos pesquisados e os valores são vetores que contem o peso em TF de cada termo nos documentos, usando como método base o método gera_docs_peso.

In [12]:
def gera_tf_vetor(frase):
    def gerador_peso(tf):
        return tf
    return gera_docs_peso(frase, gerador_peso)

# Método que gera um vetor contendo o IDF de cada termo
Este método calcula o IDF de cada termo e armazena em um vetor onde cada índice é o IDF de um dos termos.

In [13]:
def gera_idf_vetor(frase):
    termos = frase.split(" ")
    idf_vector = np.array([math.log((len(materias)+1)/len(index[termo])) for termo in termos])
    return idf_vector

# Método que gera um vetor binário da consulta
Este método verifica se cada termo existe no indice e gera uma vetor binpario onde cada indice do vetor assume o valor de 0 se não existir ou 1 se existir.

In [14]:
def gera_query_vetor(frase):
    termos = frase.split(" ")
    vetor = np.array([1 if index.get(termo) else 0 for termo in termos])
    return vetor

# Método que gera um dicionário com vetores BMF25 da consulta
Este método gera uma dicionário cujas chaves são os ids documentos que contém os termos pesquisados e os valores são vetores que contem o peso em BMF25 de cada termo nos documentos, usando como método base o método gera_tf_vetor.

In [15]:
def gera_bm25_vetor(frase):
    docs_tf = gera_tf_vetor(frase)
    k = 5
    bm25_vetor = {doc_id: np.array([((k+1)*tf)/(tf+k) for tf in tf_vetor]) for doc_id, tf_vetor in docs_tf.items()}
    return bm25_vetor

# Busca genérica
Este método encapsula as partes em comum de todos os métodos de busca implementados para evitar repetição de código.

In [16]:
def busca(frase, gerador_query, gerador_doc_vetor):
    docs_peso = gerador_doc_vetor(frase)
    query = gerador_query(frase)
    
    doc_rank = sorted(list(docs_peso.items()), key=lambda doc: np.dot(doc[1], query), reverse=True)[:5] 
    return [doc[0] for doc in doc_rank]
    

# Busca binária
Busca vetorial pelo método binário.

In [17]:
def busca_binaria(frase):
    return busca(frase, gera_query_vetor, gera_vetor_binario)

# Busca por term frequence (TF)
Busca vetorial pelo método TF.

In [18]:
def buscar_por_tf(frase):
    return busca(frase, gera_query_vetor, gera_tf_vetor)

# Busca por TF-IDF (Inverse Document Frequency)
Busca vetorial pelo método TF-IDF.

In [19]:
def buscar_por_tf_idf(frase):
    return busca(frase, gera_idf_vetor, gera_tf_vetor)

# Busca por BM25
Busca vetorial pleo método BM25.

In [20]:
def buscar_por_bm25(frase):
    return busca(frase, gera_idf_vetor, gera_bm25_vetor)

# Função para verificar precisão
Esta função verifica a precisão dos resultados retornados pelos algoritmos de busca implementados.

In [21]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# Análise de precisão da busca binária
Analizando a precisão do método de busca binária em relação a busca do google e a do gabarito.

In [22]:
busca_bin = [busca_binaria(limpar_texto(frase)) for frase in gabarito.str_busca]
print("Precisão gabarito busca binária: %.3f" %(mapk(gabarito.busca_binaria, busca_bin, k=5)))
print("Precisão gabarito busca google: %.3f" %(mapk(gabarito.google, busca_bin, k=5)))

Precisão gabarito busca binária: 0.240
Precisão gabarito busca google: 0.040


# Análise de precisão da busca por TF
Analizando a precisão do método de busca por TF em relação a busca do google e a do gabarito.

In [23]:
busca_tf = [buscar_por_tf(limpar_texto(frase)) for frase in gabarito.str_busca]
print("Precisão gabarito busca TF: %.3f" %(mapk(gabarito.tf, busca_tf, k=5)))
print("Precisão gabarito busca google: %.3f" %(mapk(gabarito.google, busca_tf, k=5)))

Precisão gabarito busca TF: 0.652
Precisão gabarito busca google: 0.048


# Análise de precisão da busca por TF-IDF
Analizando a precisão do método de busca por TF-IDF em relação a busca do google e a do gabarito.

In [24]:
busca_tfidf = [buscar_por_tf_idf(limpar_texto(frase)) for frase in gabarito.str_busca]
print("Precisão gabarito busca TF-IDF: %.3f" %(mapk(gabarito.tfidf, busca_tfidf, k=5)))
print("Precisão gabarito busca google: %.3f" %(mapk(gabarito.google, busca_tfidf, k=5)))

Precisão gabarito busca TF-IDF: 0.616
Precisão gabarito busca google: 0.058


# Análise de precisão da busca por BM25
Analizando a precisão do método de busca por BM25 em relação a busca do google e a do gabarito.

In [25]:
busca_bm25 = [buscar_por_bm25(limpar_texto(frase)) for frase in gabarito.str_busca]
print("Precisão gabarito busca BM25: %.3f" %(mapk(gabarito.bm25, busca_bm25, k=5)))
print("Precisão gabarito busca google: %.3f" %(mapk(gabarito.google, busca_bm25, k=5)))

Precisão gabarito busca BM25: 0.754
Precisão gabarito busca google: 0.128
